In [616]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.cloud import bigquery
import time
%env GOOGLE_APPLICATION_CREDENTIALS=../secrets/bigquery-service-account.json 
client = bigquery.Client()
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

env: GOOGLE_APPLICATION_CREDENTIALS=../secrets/bigquery-service-account.json


In [617]:
%load_ext google.cloud.bigquery

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [618]:
%%bigquery metadata --project masterarbeit-245718 --verbose 
SELECT * FROM `bigquery-public-data`.crypto_ethereum.INFORMATION_SCHEMA.COLUMNS where table_name = "traces"

Executing query with job ID: 6a00fcdb-4ff5-48a8-8db4-982c4731b224
Query executing: 0.70s
Query complete after 1.61s


In [619]:
size_sample_data = 200

### Generate sample data for "call_type"

In [620]:
# %%bigquery call_types_sql_result --project masterarbeit-245718 --verbose 
# select DISTINCT call_type from `bigquery-public-data.crypto_ethereum.traces`
# where DATE(block_timestamp) >= '2019-07-6' AND DATE(block_timestamp) <= '2019-7-7'

In [621]:
call_types = ['call', 'delegatecall', 'staticcall', 'callcode', None]
prob_call_types = [0.7, 0.05, 0.025, 0.025, 0.2]

In [622]:
call_type_sample = np.random.choice(call_types, 2*size_sample_data, p=prob_call_types)

# Generate sample data for "trace_type"

In [623]:
additional_trace_types = ["create", "suicide", "reward", "genesis", "daofork"]
prob_additional_trace_types = [0.025, 0.025, 0.9, 0.025, 0.025]

trace_type_sample = []
for ct in call_type_sample:
    if ct == None:
        trace_type_sample.append(np.random.choice(additional_trace_types, p=prob_additional_trace_types))
    else:
        trace_type_sample.append("call")

### Generate sample data for "status"

In [624]:
# %%bigquery status_values --project masterarbeit-245718 --verbose 
# select DISTINCT status from `bigquery-public-data.crypto_ethereum.traces`
# where DATE(block_timestamp) >= '2019-07-6' AND DATE(block_timestamp) <= '2019-7-7'

In [625]:
status_values = [0, 1]
probs_status_values = [0.05, 0.95]

In [626]:
probs_status_values = [0.05, 0.95]

In [627]:
status_sample = np.random.choice(status_values, 2*size_sample_data, p=probs_status_values)

### Generate sample accounts 

In [628]:
exchanges = ["exchange_{}".format(i) for i in range(1,int(0.05*size_sample_data + 1))]

In [629]:
speculators = ["speculator_{}".format(i) for i in range(1,int(0.95*size_sample_data + 1))]

In [630]:
addresses_testdata = []
addresses_testdata.extend(speculators)
addresses_testdata.extend(exchanges)

In [631]:
addresses_testdata = pd.DataFrame(addresses_testdata, columns=["address"])

## Upload 'addresses' table to bigquery 

In [632]:
addresses_testdata.to_gbq('ethereum_us.addresses_testdata', if_exists="replace")

1it [00:03,  3.93s/it]


### Generate sample transactions (speculators to exchanges)

In [633]:
from_spec_addresses = [np.random.choice(speculators) for i in range(int(2*size_sample_data/2))]
to_ex_addresses = [np.random.choice(exchanges) for i in range(int(2*size_sample_data/2))]
values_spec_to_ex = np.random.randint(1, 20, int(2*size_sample_data/2))

In [634]:
txdata1 = pd.DataFrame(zip(from_spec_addresses, to_ex_addresses, values_spec_to_ex), columns=["from_address", "to_address", "value"])

### Generate sample transactions (exchanges to speculators)

In [635]:
to_spec_addresses = [np.random.choice(speculators) for i in range(int(2*size_sample_data/2))]
from_ex_addresses = [np.random.choice(exchanges) for i in range(int(2*size_sample_data/2))]
values_ex_to_spec = np.random.randint(1, 5, int(2*size_sample_data/2))

In [636]:
txdata2 = pd.DataFrame(zip(from_ex_addresses, to_spec_addresses, values_ex_to_spec), columns=["from_address", "to_address", "value"])

Anmerkung: Die speculators schicken mehr Geld zu den Börsen als umgekehrt.

In [637]:
txdata = txdata1.append(txdata2).reset_index(drop=True)
txdata = txdata.sample(frac=1)

### Generate sample "block_timestamps"

In [638]:
import datetime as datetime

base = datetime.datetime.strptime("2020-02-02", '%Y-%m-%d')
block_timestamps = [base - datetime.timedelta(seconds=x) for x in range(0, 12*size_sample_data, 12)]
base = base - datetime.timedelta(days=31)
block_timestamps.extend([base - datetime.timedelta(seconds=x) for x in range(0, 12*size_sample_data, 12)])

### Merge data to sample "traces" table

In [639]:
traces_sampleData = txdata.copy()

In [640]:
traces_sampleData["status"] = status_sample 

In [641]:
traces_sampleData["call_type"] = call_type_sample 

In [642]:
traces_sampleData["trace_type"] = trace_type_sample 

In [643]:
traces_sampleData["block_timestamp"] = block_timestamps 

In [644]:
traces_sampleData.sort_values(by="from_address")

,from_address,to_address,value,status,call_type,trace_type,block_timestamp
354,exchange_1,speculator_188,2,1,call,call,2020-02-01 23:31:00
202,exchange_1,speculator_39,4,0,None,reward,2020-01-01 23:45:12
287,exchange_1,speculator_129,1,1,call,call,2020-01-01 23:59:12
383,exchange_1,speculator_106,3,1,call,call,2020-01-01 23:29:36
348,exchange_1,speculator_142,3,1,None,reward,2020-01-01 23:27:48
327,exchange_1,speculator_135,1,1,call,call,2020-02-01 23:21:00
372,exchange_1,speculator_68,3,1,call,call,2020-02-01 23:49:48
235,exchange_1,speculator_123,1,1,call,call,2020-02-01 23:52:36
279,exchange_1,speculator_15,3,1,None,reward,2020-02-01 23:42:24
232,exchange_1,speculator_165,1,1,call,call,2020-02-01 23:53:48


### Upload "traces" table to bigquery

In [645]:
traces_sampleData.to_gbq('ethereum_us.traces_sampleData', if_exists="replace")

1it [00:05,  5.12s/it]


### Test SQL command for retrieving "weiReceived", "weiSent"

In [646]:
%%bigquery res1 --project masterarbeit-245718 --verbose 
with weiView as (

  with weiReceivedView as (
        
      -- debits
      select to_address, sum(ifnull(value, 0)) as weiReceived
      from `ethereum_us.traces_sampleData` 
      where to_address is not null
      and status = 1
      and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null) 
      group by to_address
        
  ), weiSentView as (
  
      -- credits
      select from_address, sum(ifnull(value, 0)) as weiSent
      from  `ethereum_us.traces_sampleData` 
      where from_address is not null
      and status = 1
      and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null) 
      group by from_address
  ) 
  select 
  CASE 
    when to_address is not null then to_address
    when from_address is not null then from_address
  end as address, 
  ifnull(weiReceived,0) as weiReceived, 
  ifnull(weiSent,0) as weiSent
  from weiReceivedView full outer join weiSentView on from_address = to_address
) 
select address, weiReceived, weiSent from weiView right join `ethereum_us.addresses_testdata`  using(address)

Executing query with job ID: 67936e7a-6f4f-468c-82e9-be8f858b0cd0
Query executing: 1.20s
Query complete after 1.82s


In [647]:
res2 = res1.copy()
res2["balance"] = res1.weiReceived - res1.weiSent
res2 = res2.set_index("address")
res2 = res2.sort_values(by="address", ascending=False)
res2 = res2.astype("float")
res2 = res2.fillna(0.)
balance_result_sql = res2

In [648]:
%%bigquery traces_sampleData --project masterarbeit-245718 --verbose 
select * from `ethereum_us.traces_sampleData`

Executing query with job ID: 1eba48cb-c9e7-4b9e-83ba-bccf4957fd98
Query executing: 0.61s
Query complete after 1.13s


In [649]:
data1 = [row for (index, row) in traces_sampleData.iterrows() if (row.call_type not in ['delegatecall', 'callcode', 'staticcall'] or row.call_type == None) and row.status == 1]
data1 = pd.DataFrame(data1)

In [650]:
data2 = pd.DataFrame(index=set(data1["from_address"].unique()) | set(data1["to_address"].unique()))
data2["weiReceived"] = data1.groupby('to_address').sum().value
data2["weiSent"] = data1.groupby('from_address').sum().value
data2["weiSent"] = data2["weiSent"].fillna(0.)
data2["weiReceived"] = data2["weiReceived"].fillna(0.)
data2["balance"] = data2["weiReceived"] - data2["weiSent"]
data2.index = data2.index.rename("address")
data2 = data2.reindex(addresses_testdata["address"], fill_value=0.)
data2 = data2.sort_values(by="address", ascending=False)
balance_result_py = data2

In [651]:
balance_result_py.head()
balance_result_sql.head()

,weiReceived,weiSent,balance
address,,,
speculator_99,2.0,0.0,2.0
speculator_98,1.0,11.0,-10.0
speculator_97,0.0,35.0,-35.0
speculator_96,0.0,2.0,-2.0
speculator_95,1.0,0.0,1.0


,weiReceived,weiSent,balance
address,,,
speculator_99,2.0,0.0,2.0
speculator_98,1.0,11.0,-10.0
speculator_97,0.0,35.0,-35.0
speculator_96,0.0,2.0,-2.0
speculator_95,1.0,0.0,1.0


In [652]:
pd.testing.assert_frame_equal(balance_result_py, balance_result_sql)
print("weiSent, weiReceived Test succeeded!!")

weiSent, weiReceived Test succeeded!!


### Test SQL command for retrieving "txSent" and "txReceived"

In [653]:
%%bigquery res2 --project masterarbeit-245718 --verbose 
with txView as (

  with txSent as (
      SELECT from_address, count(*) as numberOfTranscationsSent FROM `masterarbeit-245718.ethereum_us.traces_sampleData` 
      where to_address is not null and status = 1 and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null) 
      group by from_address
    ), txReceived as (
      SELECT to_address, count(*) as numberOfTranscationsReceived FROM `masterarbeit-245718.ethereum_us.traces_sampleData` 
      where to_address is not null and status = 1 and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null) 
      group by to_address
    ) 
    SELECT 
    CASE  
      WHEN to_address IS NOT NULL THEN to_address
      WHEN from_address IS NOT NULL THEN from_address
    END AS address,
    IFNULL(numberOfTranscationsReceived, 0) as numberOfTranscationsReceived, 
    IFNULL(numberOfTranscationsSent, 0) as numberOfTranscationsSent
    from txReceived FULL OUTER JOIN txSent on to_address = from_address
) 

    select address, numberOfTranscationsReceived, numberOfTranscationsSent from txView right join `ethereum_us.addresses_testdata` using(address)

Executing query with job ID: 186274fd-dce6-4e47-8043-afe6b9ea76e2
Query executing: 2.29s
Query complete after 3.30s


In [654]:
res3 = res2.copy()
res3 = res3.fillna(0.)
res3 = res3.sort_values(by="address", ascending=False)
tx_sent_received_result_sql = res3.set_index("address", drop=True)

In [655]:
data3 = pd.DataFrame(index=set(data1["from_address"].unique()) | set(data1["to_address"].unique()))
data3["numberOfTranscationsReceived"] = data1.groupby('to_address').count().value
data3["numberOfTranscationsSent"] = data1.groupby('from_address').count().value
data3 = data3.fillna(0)
data3 = data3.astype("int")
data3.index = data3.index.rename("address")
data3 = data3.reindex(addresses_testdata["address"], fill_value=0.)
data3 = data3.sort_values(by="address", ascending=False)
tx_sent_received_result_py = data3

In [656]:
pd.testing.assert_frame_equal(tx_sent_received_result_py, tx_sent_received_result_sql)
print("txSent, txReceived Test succeeded !!")

txSent, txReceived Test succeeded !!


### Test avg time between tx

In [657]:
%%bigquery res4 --project masterarbeit-245718 --verbose 
with timeRecView as (

  with receivedTx as (
    SELECT to_address, count(*) as numberOfTranscationsReceived FROM `masterarbeit-245718.ethereum_us.traces_sampleData` 
    where to_address is not null 
      and status = 1 
      and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
    group by to_address),
  
  timeStampDiffs as (
    SELECT to_address, TIMESTAMP_DIFF(MAX(block_timestamp), MIN( block_timestamp ), second ) as timestampDiff
    FROM `masterarbeit-245718.ethereum_us.traces_sampleData`
    where to_address is not null 
      and status = 1 
      and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
    group by to_address
  
  ) select to_address as address, 
  CASE 
    when (numberOfTranscationsReceived - 1)  > 0 then timestampDiff / (numberOfTranscationsReceived - 1) 
    else 0
  end as avgTimeDiffBetweenReceivedTransactions
  from receivedTx inner join  timeStampDiffs using(to_address)
)

select address, ifnull(avgTimeDiffBetweenReceivedTransactions,0) as avgTimeDiffBetweenReceivedTransactions from timeRecView right join `ethereum_us.addresses_testdata` using(address)

Executing query with job ID: 40f713e3-f961-43c3-a0be-1eb12066d1c7
Query executing: 1.61s
Query complete after 2.34s


In [658]:
res6 = res4.set_index("address", drop=True)
res6 = res6.fillna(0.)
res6 = res6.sort_values(by="address")
avg_time_diff_receivedtx_result_sql = res6

In [659]:
%%bigquery res5 --project masterarbeit-245718 --verbose 
with timeSentView as (

  with sentTx as (
    SELECT from_address, count(*) as numberOfTranscationsSent FROM `masterarbeit-245718.ethereum_us.traces_sampleData` 
    where to_address is not null 
      and status = 1 
      and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
    group by from_address),
  timeStampDiffs as (
    SELECT from_address, TIMESTAMP_DIFF(MAX(block_timestamp), MIN( block_timestamp ), second ) as timestampDiff
    FROM `masterarbeit-245718.ethereum_us.traces_sampleData`
    where to_address is not null 
      and status = 1 
      and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
    group by from_address
  ) select from_address as address, 
  CASE 
    when (numberOfTranscationsSent - 1)  > 0 then timestampDiff / (numberOfTranscationsSent - 1) 
    else 0
  end as avgTimeDiffBetweenSentTransactions
     from sentTx inner join  timeStampDiffs using(from_address)
)

select address, ifnull(avgTimeDiffBetweenSentTransactions,0) as avgTimeDiffBetweenSentTransactions from timeSentView right join `ethereum_us.addresses_testdata` using(address)

Executing query with job ID: 2e1e2c6a-792e-410c-a003-107879b729f0
Query executing: 1.14s
Query complete after 1.76s


In [660]:
res6 = res5.set_index("address", drop=True)
res6 = res6.fillna(0.)
res6 = res6.sort_values(by="address")
avg_time_diff_senttx_result_sql = res6

In [661]:
res7 = [row for (index, row) in traces_sampleData.iterrows() if (row.call_type not in ['delegatecall', 'callcode', 'staticcall'] or row.call_type == None) and row.status == 1]
res7 = pd.DataFrame(res7)
res8 = res7.groupby("from_address").max().block_timestamp
res9 = res7.groupby("from_address").min().block_timestamp
res10 = res8 - res9
res10 = res10.rename("seconds_diff")
res10 = tx_sent_received_result_py.join(res10, how="right").drop("numberOfTranscationsReceived", axis=1)
res10 = res10.fillna(0.)
res10["avgTimeDiffBetweenSentTransactions"] = res10["seconds_diff"] / (res10["numberOfTranscationsSent"] - 1)
res10["avgTimeDiffBetweenSentTransactions"] = res10["avgTimeDiffBetweenSentTransactions"].fillna(datetime.timedelta(0))
res10 = res10.reindex(addresses_testdata["address"], fill_value=0.)
res10.index = res10.index.rename("address")
res10 = res10.sort_values(by="address")
res10 = res10.drop(["numberOfTranscationsSent", "seconds_diff"],axis=1)
res10["avgTimeDiffBetweenSentTransactions"] = [ts.total_seconds() for ts in res10["avgTimeDiffBetweenSentTransactions"]]
avg_time_diff_senttx_result_py = res10

/home/mgsgde/.local/lib/python3.6/site-packages/pandas/core/arrays/timedeltas.py:523: RuntimeWarning: invalid value encountered in true_divide
  result = self._data / other


In [662]:
pd.testing.assert_frame_equal(avg_time_diff_senttx_result_sql, avg_time_diff_senttx_result_py)
print("avgTimeDiffBetweenSentTransactions Test succeeded !!")

avgTimeDiffBetweenSentTransactions Test succeeded !!


In [663]:
features = balance_result_sql.join(tx_sent_received_result_sql,how="left")
features = features.join(avg_time_diff_senttx_result_sql)
features = features.join(avg_time_diff_receivedtx_result_sql)
features = features.sort_values(by="balance", ascending=False)
features = features.fillna(0.0)
import sys
# addresses that have sent/received only one transaction get the avg time max * 2
features["avgTimeDiffBetweenSentTransactions"] = features["avgTimeDiffBetweenSentTransactions"].replace(to_replace=0.0, value=2 * max(features["avgTimeDiffBetweenSentTransactions"]))
features["avgTimeDiffBetweenReceivedTransactions"] = features["avgTimeDiffBetweenReceivedTransactions"].replace(to_replace=0.0, value=2 * max(features["avgTimeDiffBetweenReceivedTransactions"]))

In [664]:
features = features.reset_index()

In [665]:
features.to_gbq('ethereum_us.features_sampleData', if_exists="replace")

1it [00:04,  4.68s/it]


# Test mined Blocks

In [666]:
%%bigquery sql_res_minedblocks --project masterarbeit-245718 --verbose 

with minedBlocksView as (
    SELECT to_address as address, count(*) as mined_blocks FROM `masterarbeit-245718.ethereum_us.traces_sampleData`
        where trace_type = "reward"
        group by to_address
    )
select address, ifnull(mined_blocks,0) as minedBlocks from minedBlocksView right join `ethereum_us.addresses_testdata` using(address)


Executing query with job ID: 34b13ea5-ccab-4115-9523-a98737655c5d
Query executing: 0.63s
Query complete after 1.56s


In [667]:
features.head()

,address,weiReceived,weiSent,balance,numberOfTranscationsReceived,numberOfTranscationsSent,avgTimeDiffBetweenSentTransactions,avgTimeDiffBetweenReceivedTransactions
0,exchange_4,277.0,55.0,222.0,27.0,22.0,127651.428571,103103.076923
1,exchange_10,217.0,36.0,181.0,24.0,17.0,167538.000000,116529.913043
2,exchange_5,215.0,39.0,176.0,23.0,15.0,191473.714286,121839.272727
3,exchange_1,194.0,29.0,165.0,18.0,14.0,206150.769231,157615.764706
4,exchange_7,184.0,28.0,156.0,17.0,14.0,206190.461538,167539.500000


# Test var(avgTimeDiff...)

In [668]:
%%bigquery res_sql_stddev --project masterarbeit-245718 --verbose 

with timestamp_var as (
    with timestamps_diffs as (
        
        with timestamps_preceding_tx as (
            
            with timestamps_sent_tx as (
                select from_address, block_timestamp from `masterarbeit-245718.ethereum_us.traces_sampleData`
                    where from_address is not null 
                      and status = 1 
                      and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
            )
            
            select from_address, block_timestamp,
                lag(block_timestamp) OVER (partition by from_address order by block_timestamp asc) as preceding_block_timestamp 
            from timestamps_sent_tx
        )
        
        select from_address, block_timestamp, preceding_block_timestamp, 
            TIMESTAMP_DIFF(block_timestamp, preceding_block_timestamp, second) as timestampdiff
        from timestamps_preceding_tx
    )
    select from_address as address, STDDEV_SAMP(timestampdiff) as stddev_sample  
    from timestamps_diffs group by from_address 
) 
select * from timestamp_var right join `ethereum_us.addresses_testdata` using(address)

Executing query with job ID: 1793389f-a940-4722-8134-055491d97e45
Query executing: 1.16s
Query complete after 1.69s


In [669]:
stddev_result_sql = res_sql_stddev.set_index("address")
stddev_result_sql = stddev_result_sql.reindex(addresses_testdata["address"])
stddev_result_sql = stddev_result_sql.fillna(res_sql_stddev["stddev_sample"].max())
stddev_result_sql = stddev_result_sql.sort_values(by="address")

In [670]:
%%bigquery res_sql_stddev --project masterarbeit-245718 --verbose 

with timestamp_var as (
    
    with timestamps_diffs as (
        
        with timestamps_preceding_tx as (
            
            with timestamps_sent_tx as (
                select to_address, block_timestamp from `masterarbeit-245718.ethereum_us.traces_sampleData`
                    where to_address is not null 
                      and status = 1 
                      and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null)
            )
            
            select to_address, block_timestamp,
                lag(block_timestamp) OVER (partition by to_address order by block_timestamp asc) as preceding_block_timestamp 
            from timestamps_sent_tx
        )
        
        select to_address, block_timestamp, preceding_block_timestamp, 
            TIMESTAMP_DIFF(block_timestamp, preceding_block_timestamp, second) as timestampdiff
        from timestamps_preceding_tx
    )
    select to_address as address, STDDEV_SAMP(timestampdiff) as stddev_sample  
    from timestamps_diffs group by to_address 
) 
select * from timestamp_var right join `ethereum_us.addresses_testdata` using(address)

Executing query with job ID: 9a367f4b-f735-40aa-9776-ecd588bf8a35
Query executing: 1.21s
Query complete after 1.93s


In [671]:
stddev_result_sql = res_sql_stddev.set_index("address")
stddev_result_sql = stddev_result_sql.reindex(addresses_testdata["address"])
stddev_result_sql = stddev_result_sql.fillna(res_sql_stddev["stddev_sample"].max())
stddev_result_sql = stddev_result_sql.sort_values(by="stddev_sample")

# Verifikation der var(avgTimeDiff...) Tabelle via Python

In [672]:
res7 = [row for (index, row) in traces_sampleData.iterrows() if (row.call_type not in ['delegatecall', 'callcode', 'staticcall'] or row.call_type == None) and row.status == 1]
res7 = pd.DataFrame(res7)
res7 = res7.sort_values(by="block_timestamp", ascending=True)

In [673]:
def calculateDifference(df):
#     from_address = df["from_address"].reset_index(drop=True)[0]
    variance = df["block_timestamp"].diff().std()
    return variance
#     return pd.DataFrame({"variance": [variance]})

res8 = res7.groupby("to_address").apply(lambda df: df["block_timestamp"].diff().std() )
res8 = pd.DataFrame(res8, columns=["stddev_sample"])
res8 = res8.reindex(addresses_testdata["address"])
res8 = res8.fillna(res8.max())
res8["stddev_sample"] = res8["stddev_sample"] / np.timedelta64(1, 's')
res8 = res8.sort_values(by="stddev_sample")
stddev_result_py = res8

In [674]:
# pd.testing.assert_frame_equal(stddev_result_py, stddev_result_sql)
np.testing.assert_almost_equal(list(stddev_result_py["stddev_sample"]), list(stddev_result_sql["stddev_sample"]), decimal=2)
print("stddev Test succeeded !!")

stddev Test succeeded !!


# Erstellen der ETH USD Tabelle

In [675]:
import os 
from datetime import datetime

In [676]:
eth_usd_res = pd.read_csv('../data/eth_usd.csv') 
# Preview the first 5 lines of the loaded data 
eth_usd = eth_usd_res.copy()
eth_usd["avg_usd_eth"] = (eth_usd_res["High"] + eth_usd_res["Low"]) / 2
eth_usd["usd_eth_timestamp"] = [datetime.strptime(ts, "%Y-%m-%d") for ts in eth_usd_res["Date"]]
eth_usd = eth_usd.loc[:, ["usd_eth_timestamp", "avg_usd_eth"]]
# data.sort_values(by="avg", ascending=False)
# data.iloc[data["avg_usd_eth"].idxmax(),:]
eth_usd.tail()

,usd_eth_timestamp,avg_usd_eth
1645,2020-02-07,217.722564
1646,2020-02-08,220.987190
1647,2020-02-09,226.423241
1648,2020-02-10,223.632332
1649,2020-02-11,222.864899


In [677]:
eth_usd.to_gbq('ethereum_us.usd_eth', if_exists="replace")

1it [00:06,  6.45s/it]


In [678]:
%%bigquery traces_usd --project masterarbeit-245718 --verbose 

select * from `masterarbeit-245718.ethereum_us.traces_sampleData` as traces left join `masterarbeit-245718.ethereum_us.usd_eth` as usd_eth 
  on (TIMESTAMP_TRUNC(usd_eth.usd_eth_timestamp, DAY, 'UTC') = TIMESTAMP_TRUNC(traces.block_timestamp, DAY, 'UTC'))

Executing query with job ID: 23971b88-6ce8-48df-9b4d-870df13336cf
Query executing: 1.69s
Query complete after 2.52s


In [679]:
traces_usd.to_gbq('ethereum_us.traces_usd_sampleData', if_exists="replace")

1it [00:03,  3.51s/it]


# Active Months

In [694]:
%%bigquery --project masterarbeit-245718 --verbose 

with traces_clean as (
    select * from `masterarbeit-245718.ethereum_us.traces_sampleData` where 
    status = 1 and (call_type not in ('delegatecall', 'callcode', 'staticcall') or call_type is null) 
    
), tx_received as (
    select 
        TIMESTAMP_TRUNC(block_timestamp, MONTH, 'UTC') as month, 
        to_address,
        count(*) as number_tx_received
    from traces_clean
    group by TIMESTAMP_TRUNC(block_timestamp, MONTH, 'UTC'), to_address
    
), tx_sent as (
    select 
        TIMESTAMP_TRUNC(block_timestamp, MONTH, 'UTC') as month, 
        from_address,
        count(*) as number_tx_sent
    from traces_clean
    group by TIMESTAMP_TRUNC(block_timestamp, MONTH, 'UTC'), from_address
    
), monthly_tx as (
    select 
        CASE  
          WHEN tx_sent.from_address IS NOT NULL THEN tx_sent.from_address
          WHEN tx_received.to_address IS NOT NULL THEN tx_received.to_address
        END AS address,
        CASE  
          WHEN tx_sent.month IS NOT NULL THEN tx_sent.month
          WHEN tx_received.month IS NOT NULL THEN tx_received.month
        END AS month,
        ifnull(number_tx_sent,0) as number_tx_sent, 
        ifnull(number_tx_received,0) as number_tx_received 
    from tx_sent full join tx_received 
        on (tx_sent.from_address = tx_received.to_address and tx_sent.month = tx_received.month)

), active_months_view as (
    select 
        address, 
        countif(number_tx_sent > 0 or number_tx_received > 0) as active_months 
    from monthly_tx group by address order by address ASC 
)

select 
    address, 
    ifnull(active_months,0) as active_months 
from ethereum_us.addresses_testdata left join active_months_view using(address)



Executing query with job ID: d991bb85-937e-42d0-bf14-25ce7e4cda67
Query executing: 1.82s
Query complete after 2.96s


,address,active_months
0,speculator_1,1
1,speculator_2,1
2,speculator_3,1
3,speculator_4,2
4,speculator_5,2
5,speculator_6,2
6,speculator_7,1
7,speculator_8,2
8,speculator_9,2
9,speculator_10,0


In [681]:
%%bigquery --project masterarbeit-245718 --verbose 

select EXTRACT(MONTH FROM block_timestamp) as month, count(*) as number_of_traces from `masterarbeit-245718.ethereum_us.traces_usd_sampleData` group by month

Executing query with job ID: 98ede515-7b12-4742-aa81-2230ee120a19
Query executing: 0.69s
Query complete after 1.61s


,month,number_of_traces
0,1,200
1,2,200


In [682]:
%%bigquery --project masterarbeit-245718 --verbose 

select *, 
    count(*) over (partition by block_timestamp, from_address) as count
from `masterarbeit-245718.ethereum_us.traces_usd_sampleData` order by block_timestamp

Executing query with job ID: 3238b201-aaa5-43b8-9866-fa6c5792abda
Query executing: 0.57s
Query complete after 1.40s


,from_address,to_address,value,status,call_type,trace_type,block_timestamp,usd_eth_timestamp,avg_usd_eth,count
0,exchange_3,speculator_47,4,1,call,call,2020-01-01 23:20:12+00:00,2020-01-01 00:00:00+00:00,131.016823,1
1,speculator_163,exchange_4,2,1,None,reward,2020-01-01 23:20:24+00:00,2020-01-01 00:00:00+00:00,131.016823,1
2,exchange_2,speculator_34,1,1,call,call,2020-01-01 23:20:36+00:00,2020-01-01 00:00:00+00:00,131.016823,1
3,exchange_5,speculator_51,3,1,call,call,2020-01-01 23:20:48+00:00,2020-01-01 00:00:00+00:00,131.016823,1
4,exchange_10,speculator_83,3,1,call,call,2020-01-01 23:21:00+00:00,2020-01-01 00:00:00+00:00,131.016823,1
5,exchange_4,speculator_89,4,1,call,call,2020-01-01 23:21:12+00:00,2020-01-01 00:00:00+00:00,131.016823,1
6,speculator_96,exchange_8,2,1,call,call,2020-01-01 23:21:24+00:00,2020-01-01 00:00:00+00:00,131.016823,1
7,exchange_3,speculator_176,2,1,call,call,2020-01-01 23:21:36+00:00,2020-01-01 00:00:00+00:00,131.016823,1
8,exchange_8,speculator_90,1,1,call,call,2020-01-01 23:21:48+00:00,2020-01-01 00:00:00+00:00,131.016823,1
9,exchange_7,speculator_57,2,1,call,call,2020-01-01 23:22:00+00:00,2020-01-01 00:00:00+00:00,131.016823,1


# Deprecated

In [683]:
# %%bigquery res6 --project masterarbeit-245718 --verbose 

# SELECT
# * 
# FROM
# `masterarbeit-245718.ethereum_us.INFORMATION_SCHEMA.TABLES`


In [684]:
# tablesToDelete = [tn for tn in res6["table_name"] if tn.startswith("top40k_19_20")]

# for t in tablesToDelete:
#      client.delete_table("ethereum_us.{}".format(t), not_found_ok=True)  

In [685]:
# view_ref = shared_dataset_ref.table("my_shared_view")
# view = bigquery.Table(view_ref)
# sql_template = 'SELECT name, post_abbr FROM `{}.{}.{}` WHERE name LIKE "W%"'
# view.view_query = sql_template.format(project, source_dataset_id, source_table_id)
# view = client.create_table(view)  # API request

# print("Successfully created view at {}".format(view.full_table_id))